In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import numpy as np
import dotenv
import os
from sentinelhub import SHConfig
import logging

logging.basicConfig(level=logging.DEBUG)

Configuration

In [ ]:
dotenv.load_dotenv()

# Get client credentials from environment variables
clientID = os.getenv("clientID")
clientSecret = os.getenv("clientSecret")

config = SHConfig()
config.instance_id = "test"
config.sh_client_id = clientID
config.sh_client_secret = clientSecret
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
config.save()